In [16]:
%run "../00_project_config.ipynb"

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib as pl

sub_stats = pl.Path(
    "/home/ebertp/work/projects/hgsvc/2023_assm_stats/verkko",
    "assembly_stats_sub.verkko.sseq.tsv"
)

stats = pd.read_csv(sub_stats, header=0, sep="\t", comment="#")
stats["sex"] = stats["sample"].replace(
    dict((row.sample, row.sex) for row in HGSVC_SAMPLES[["sample", "sex"]].itertuples())
)

select_unit = stats["assembly_unit"] == "unassigned"
select_stat = stats["statistic"] == "total_length_grt_0bp"
select_unps = stats.loc[select_unit & select_stat, "value"] < int(1e8)
select_samples1 = set(stats.loc[select_unps.index[select_unps], "sample"].values)

select_unit = stats["assembly_unit"] == "disconnected"
select_stat = stats["statistic"] == "total_length_grt_0bp"
select_disc = stats.loc[select_unit & select_stat, "value"] < int(1e7)
select_samples2 = set(stats.loc[select_disc.index[select_disc], "sample"].values)

select_sub = select_samples1.intersection(select_samples2)

#print(stats.loc[(stats["sample"].isin(select_sub)) & (stats["sex"] == "female"), "sample"].unique())

#raise

select_stat1 = stats["statistic"] == "length_auN_grt_0bp"
select_stat2 = stats["statistic"] == "total_num_grt_0bp"
select_stat3 = stats["statistic"] == "total_length_grt_0bp"
select_sex = stats["sex"] == "female"
select_unit = stats["assembly_unit"] == "disconnected"
selector = select_stat3 & select_sex & select_unit

print(stats.loc[selector, :])
raise

statistic = ["length_auN_grt_0bp", "total_num_grt_0bp", "total_length_grt_0bp"]
stat_labels = {
    "length_auN_grt_0bp": "auN",
    "total_num_grt_0bp": "#seq",
    "total_length_grt_0bp": "length"
}
asm_units = ["asm-hap1", "asm-hap2", "unassigned", "disconnected", "rdna"]
au_labels = {
    "asm-hap1": "hap1",
    "asm-hap2": "hap2",
    "unassigned": "unps",
    "disconnected": "single",
    "rdna": "rDNA"
}

for s in statistic:
    stat_label = stat_labels[s]
    
    plot_large = []
    plot_small = []
    
    large_labels = []
    small_labels = []
    y_unit_large = None
    y_unit_small = None
    for sex, infos in stats.groupby("sex"):
        
        for au in asm_units:
            au_label = au_labels[au]
            num_samples = infos["sample"].nunique()
            label = f"{stat_label}\n{au_label}\n{sex}\n(n={num_samples})"
            select_stat = infos["statistic"] == s
            select_au = infos["assembly_unit"] == au
            rows = select_stat & select_au
            assert rows.any()
            data = infos.loc[rows, "value"].values
            
            if au_label in ["hap1", "hap2", "unps"]:
                
                if "seq" in stat_label:
                    y_unit_large = "N"
                elif data.max() > 1e6:
                    data /= 1e6
                    y_unit_large = "Mbp"
                else:
                    y_unit_large = "bp"
                    
                plot_large.append(data)
                large_labels.append(label)
                
            else:
                
                if "seq" in stat_label:
                    y_unit_small = "N"
                elif data.max() > 1e6:
                    data /= 1e6
                    y_unit_small = "Mbp"
                elif data.min() > 1e3:
                    data /= 1e3
                    y_unit_small = "kbp"
                else:
                    y_unit_small = "bp"

                plot_small.append(data)
                small_labels.append(label)
                
    fig, ax = plt.subplots(figsize=(8,8))
    ax.boxplot(
        plot_large,
        labels=large_labels
    )
    ax.set_ylabel(f"{y_unit_large}", fontsize=14)
    
    fig, ax = plt.subplots(figsize=(8,8))
    ax.boxplot(
        plot_small,
        labels=small_labels
    )
    ax.set_ylabel(f"{y_unit_small}", fontsize=14)

    
            
        
        
        

['HG00268' 'HG00733' 'HG01352' 'HG02059' 'HG02106' 'HG02769' 'HG02818'
 'HG03452' 'HG04036' 'HG04217' 'NA19129' 'NA19434' 'NA19836' 'NA21487']


RuntimeError: No active exception to reraise